![clothing_classification](clothing_classification.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [4]:
# Run the cells below first

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [6]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Resize((64,64))]))
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Resize((64,64))]))

In [7]:
train_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
           )

In [8]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, shuffle=False)

In [9]:
# Start coding here
# Use as many cells as you need

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.extractor = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size = 2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size = 2),            
        )
        self.fc = nn.Sequential(nn.Linear(64*16*16, 128), nn.ELU(),nn.Linear(128,10))
        
    def forward(self, x):
        x = self.extractor(x)
        x = x.view(x.size(0), -1) #flatten
        x = self.fc(x)
        return x
        


In [10]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr =0.001)
acc = Accuracy(task="multiclass", num_classes=10)
prc = Precision(task="multiclass", num_classes=10, average=None)
rec = Recall(task="multiclass", num_classes=10, average=None)


In [11]:
for epoch in range(1):
    for image, label in train_dataloader:
        optimizer.zero_grad()
        output = net(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        

In [12]:
#test
predictions = []
net.eval()
with torch.no_grad():
    for images, labels in test_dataloader:
        outputs = net(images)
        _, preds = torch.max(outputs, 1)
        predictions.append(preds)
        acc(preds, labels)
        prc(preds, labels)
        rec(preds, labels)
accuracy = float(acc.compute())
precision = prc.compute()
recall = rec.compute()

    

In [13]:
precision = precision.tolist()
recall = recall.tolist()